In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/YOURPATH/

In [13]:
from utils import h36motion3d as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch.optim as optim
import torch.autograd
import torch
import numpy as np
from utils.loss_funcs import *
from utils.data_utils import define_actions
from utils.h36_3d_viz import visualize
import time
from tqdm import tqdm

import torch.nn.functional as F

In [ ]:
# Use GPU if available, otherwise stick with cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device,  '- Type:', torch.cuda.get_device_name(0))

In [2]:
# # Arguments to setup the datasets
datas = 'h36m' # dataset name
path = './data/h3.6m/h3.6m/dataset'
input_n=10 # number of frames to train on (default=10)
output_n=25 # number of frames to predict on
input_dim=3 # dimensions of the input coordinates(default=3)
skip_rate=1 # # skip rate of frames
joints_to_consider=22


#FLAGS FOR THE TRAINING
mode='train' #choose either train or test mode

batch_size_test=8
model_path= './checkpoints/' # path to the model checkpoint file

actions_to_consider_test='all' # actions to test on.
model_name = datas+'_3d_'+str(output_n)+'frames_ckpt' #the model name to save/load

#FLAGS FOR THE VISUALIZATION
actions_to_consider_viz='all' # actions to visualize
visualize_from='test'
n_viz=2

In [3]:
# Load Data
print('Loading Train Dataset...')
dataset = datasets.Datasets(path,input_n,output_n,skip_rate, split=0)
print('Loading Validation Dataset...')
vald_dataset = datasets.Datasets(path,input_n,output_n,skip_rate, split=1)

#! Note: Ignore warning:  "VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences"

Loading Train Dataset...


c:\Users\aless\OneDrive\Desktop\Projects\advanced-machine-learning\Homework 3\Practice\utils\h36motion3d.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  subs = np.array([[1, 6, 7, 8, 9], [11], [5]])  # , 6, 7, 8, 9


Loading Validation Dataset...


In [5]:
batch_size=256

print('>>> Training dataset length: {:d}'.format(dataset.__len__()))
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)#

print('>>> Validation dataset length: {:d}'.format(vald_dataset.__len__()))
vald_loader = DataLoader(vald_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

>>> Training dataset length: 180077
>>> Validation dataset length: 28110


In [6]:
from models.sttr.sttformer import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

n_heads = 1

def instantiate_model(num_joints=joints_to_consider, num_frames=input_n, num_frames_out=output_n, num_heads=n_heads, num_channels=3, kernel_size=[3,3], use_pes=True):
    model = Model(num_joints, num_frames, num_frames_out, num_heads, num_channels, kernel_size, use_pes).to(device)
    print('total number of parameters of the network is: '+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    return model

Using device: cpu


In [7]:
clip_grad=None # select max norm to clip gradients
# Argument for training
n_epochs=41
log_step = 200

In [8]:
import os
from os import path

def train(model, data_loader, vald_loader, lr, ml, wd, gamma, use_scheduler=True, save_and_plot=True, path_to_save_model=None, path_to_save_plots=None):
    optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=wd)

    if use_scheduler:
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=ml, gamma=gamma)

    # Creation of a folder where to save plots
    if path.exists(path_to_save_plots) == False:
        os.mkdir(path_to_save_plots)

    # Creation of a folder where to save models
    if path.exists(path_to_save_model) == False:
        os.mkdir(path_to_save_model)

    train_loss = []
    val_loss = []
    val_loss_best = 1000

    dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                        26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                        46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                        75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])

    for epoch in range(n_epochs-1):
        running_loss=0
        n=0
        model.train()
        for cnt,batch in enumerate(data_loader):
            batch=batch.float().to(device)
            batch_dim=batch.shape[0]
            n+=batch_dim

            sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
            sequences_gt=batch[:, input_n:input_n+output_n, dim_used].view(-1,output_n,len(dim_used)//3,3)

            optimizer.zero_grad()
            sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)


            loss=mpjpe_error(sequences_predict,sequences_gt)


            if cnt % log_step == 0:
                print('[Epoch: %d, Iteration: %5d]  training loss: %.3f' %(epoch + 1, cnt + 1, loss.item()))

            loss.backward()
            if clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),clip_grad)

            optimizer.step()
            running_loss += loss*batch_dim

        train_loss.append(running_loss.detach().cpu()/n)
        model.eval()
        with torch.no_grad():
            running_loss=0
            n=0
            for cnt,batch in enumerate(vald_loader):
                batch=batch.float().to(device)
                batch_dim=batch.shape[0]
                n+=batch_dim


                sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
                sequences_gt=batch[:, input_n:input_n+output_n, dim_used].view(-1,output_n,len(dim_used)//3,3)

                sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)
                loss=mpjpe_error(sequences_predict,sequences_gt)

                if cnt % log_step == 0:
                            print('[Epoch: %d, Iteration: %5d]  validation loss: %.3f' %(epoch + 1, cnt + 1, loss.item()))
                running_loss+=loss*batch_dim
            val_loss.append(running_loss.detach().cpu()/n)
            if running_loss/n < val_loss_best:
                val_loss_best = running_loss/n

        if use_scheduler:
            scheduler.step()

        # save and plot model every 5 epochs
        '''
        Insert your code below. Use the argument path_to_save_model to save the model to the path specified.
        '''
        if save_and_plot and (epoch+1)%5==0 and epoch!=0: # Save the model and display the losses every 5 epochs
            torch.save(model.state_dict(), f'{path_to_save_model}/h36m_3d_25frames_ckpt_epoch_{epoch+1}_q1.pt')

            # Plot the training and validation loss
            fig, ax = plt.subplots()
            x_tick_freq = 1 if epoch < 16 else 2 if epoch<26 else 4
            epochs = range(1, epoch+2, x_tick_freq) # The epoch numbering in the plots starts from 1 in order to stick with the code above
            x_list = list(range(1, len(train_loss)+1))
            ax.plot(x_list, train_loss, 'r', label='Train loss') # Line that displays the train loss
            ax.plot(x_list, val_loss, 'g', label='Val loss') # Line that displays the validation loss
            params = "Hyperparams: lr-> " + str(lr) + ", ml->" + str(ml) + ", wd->" + str(wd)
            ax.set_xticks(list(epochs))
            ax.set_xlabel('Epochs \n ' + params)
            ax.set_ylabel('Loss')
            plt.legend()
            plt.title('Loss History', fontsize=12)

            plt.savefig(path_to_save_plots + "/loss_epoch_"+str(epoch+1)+"-"+str(lr)+"-"+str(ml)+"-"+str(wd)+".png", bbox_inches='tight')
            plt.show()

    return train_loss, val_loss

In [14]:
learning_rates = [1e-1, 1e-2]
milestones = [[10, 25, 30], [15, 25, 35], [15, 30]]
weight_decays = [0, 1e-2, 1e-5]

all_hp_combinations = [(lr, ml, wd) for lr in learning_rates for ml in milestones for wd in weight_decays]

best_validation_loss = float('inf')
best_hyperparameters = None

path_to_save_model = './HyperparamsTuning/models'
path_to_save_plots = './HyperparamsTuning/plots'

# Creation of a folder where to save plots
if path.exists("./HyperparamsTuning/") == False:
    os.mkdir("./HyperparamsTuning/")

for lr, ml, wd in tqdm(all_hp_combinations):
    model = instantiate_model() # All default model parameters are chosen
    training_losses, validation_losses = train(model, data_loader, vald_loader, lr, ml, wd, gamma=0.1, use_scheduler=True, save_and_plot=True, path_to_save_model=path_to_save_model, path_to_save_plots=path_to_save_plots)

    # Calculate the minimum validation loss
    min_validation_loss = min(validation_losses)

    if min_validation_loss < best_validation_loss:
        best_validation_loss = min_validation_loss
        best_hyperparameters = (lr, ml, wd)

print(f"Best validation loss reached: {best_validation_loss}")
print(f"Best hyperparameters: {best_hyperparameters}")

  0%|          | 0/18 [00:00<?, ?it/s]

total number of parameters of the network is: 26859
[Epoch: 1, Iteration:     1]  training loss: 558.251


  0%|          | 0/18 [00:04<?, ?it/s]


KeyboardInterrupt: 